# How to use pyELQ
This example is intended to provide a basic overview on how to work with the pyELQ repo. We will set up a basic example where we generate some concentration data and try to estimate the source location and emisson rate of these synthetic sources.

First we import all the required packages.

In [2]:
import datetime
from copy import deepcopy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pyelq.component.background import SpatioTemporalBackground
from pyelq.component.error_model import BySensor
from pyelq.component.offset import PerSensor
from pyelq.component.source_model import Normal
from pyelq.coordinate_system import ENU, LLA
from pyelq.dispersion_model.gaussian_plume import GaussianPlume
from pyelq.gas_species import CH4
from pyelq.model import ELQModel
from pyelq.meteorology import Meteorology
from pyelq.plotting.plot import Plot
from pyelq.preprocessing import Preprocessor
from pyelq.sensor.beam import Beam
from pyelq.sensor.sensor import Sensor, SensorGroup
from pyelq.source_map import SourceMap

Next we set up a time axis as well as a reference point which we use in our local coordinate systems.

In [3]:
time_axis = pd.array(pd.date_range(start="2024-01-01 08:00:00", end="2024-01-01 12:00:00", freq="120s"), dtype='datetime64[ns]')
nof_observations = time_axis.size
reference_latitude = 0
reference_longitude = 0
reference_altitude = 0

We define a couple of regularly spaced beam sensors by creating a local ENU frame, defining the beam end points in that coordinate frame and next transform the ENU coordinates to LLA coordinates.

In [4]:
radius = 30
angles = np.linspace(0, 90, 5)
sensor_x = radius * np.cos(angles*np.pi/180)
sensor_y = radius * np.sin(angles*np.pi/180)
sensor_z = np.ones_like(sensor_x) * 5.0

In [5]:
sensor_y

array([ 0.        , 11.48050297, 21.21320344, 27.71638598, 30.        ])

In [6]:
ENU_object = ENU(ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude)
ENU_object.from_array(np.vstack([sensor_x, sensor_y, sensor_z]).T)
LLA_object = ENU_object.to_lla()
LLA_array = LLA_object.to_array()
print(LLA_array)

[[0.00000000e+00 2.69494374e-04 5.00007055e+00]
 [1.03826083e-04 2.48980336e-04 5.00007062e+00]
 [1.91845586e-04 1.90561299e-04 5.00007079e+00]
 [2.50658337e-04 1.03131032e-04 5.00007096e+00]
 [2.71310629e-04 1.65017711e-20 5.00007103e+00]]


We create a SensorGroup which contains all the 5 beams we have set up. We set the sensor position (beam start points) to be at the reference latitude and longitude. At an altitude of 3 meters, similar to the beam end points. The beam layout can be seen in the plot. We initialize the concentration and the time attributes of the sensor so we can use it later to calculate the simulated concentration observations.

In [7]:
nof_sensors = LLA_array.shape[0]
sensor_group = SensorGroup()
for sensor in range(nof_sensors):
    new_sensor = Beam()
    new_sensor.label = f"Beam sensor {sensor}"
    new_sensor.location = LLA(
        latitude=np.array([reference_latitude, LLA_object.latitude[sensor]]),
        longitude=np.array([reference_longitude, LLA_object.longitude[sensor]]),
        altitude=np.array([5.0, LLA_object.altitude[sensor]])
    )
    
    new_sensor.time = time_axis
    new_sensor.concentration = np.zeros(nof_observations)
    sensor_group.add_sensor(new_sensor)

In [8]:
nof_sensors

5

In [9]:
sensor_group.location.latitude

array([0.        , 0.        , 0.        , 0.00010383, 0.        ,
       0.00019185, 0.        , 0.00025066, 0.        , 0.00027131])

Let's also add some point sensors to our SensorGroup

In [10]:
sensor_x = np.array([5, 20])
sensor_y = np.array([22, 5])
sensor_z = np.ones_like(sensor_x) * 1.0
ENU_object = ENU(ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude)
ENU_object.from_array(np.vstack([sensor_x, sensor_y, sensor_z]).T)
LLA_object = ENU_object.to_lla()
LLA_array = LLA_object.to_array()

nof_sensors = LLA_array.shape[0]
for sensor in range(nof_sensors):
    new_sensor = Sensor()
    new_sensor.label = f"Point sensor {sensor}"
    new_sensor.location = LLA(
        latitude=np.array([LLA_object.latitude[sensor]]),
        longitude=np.array([LLA_object.longitude[sensor]]),
        altitude=np.array([LLA_object.altitude[sensor]])
    )
    
    new_sensor.time = time_axis
    new_sensor.concentration = np.zeros(nof_observations)
    sensor_group.add_sensor(new_sensor)

In [11]:
fig = go.Figure()
fig = sensor_group.plot_sensor_location(fig=fig)
fig.update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
fig.show(renderer="browser")

We use the meteorology object to store the simulated meteorology observations like wind speed and direction and show these in a wind rose plot.

In [12]:
met_object = Meteorology()

met_object.time = time_axis
met_object.wind_direction = np.linspace(0.0, 90.0, nof_observations) + np.random.normal(loc=0.0, scale=0.1, size=nof_observations)
met_object.wind_speed = 4.0 * np.ones_like(met_object.wind_direction) + np.random.normal(loc=0.0, scale=0.1, size=nof_observations)

met_object.calculate_uv_from_wind_speed_direction()

met_object.temperature = (273.1 + 15.0) * np.ones_like(met_object.wind_direction)
met_object.pressure = 101.325 * np.ones_like(met_object.wind_direction)

met_object.wind_turbulence_horizontal = 5.0 * np.ones_like(met_object.wind_direction)
met_object.wind_turbulence_vertical = 5.0 * np.ones_like(met_object.wind_direction)

fig = met_object.plot_polar_hist()
fig.update_layout(height=400, margin={"r":0,"l":0})
fig.show(renderer="browser")

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_axis, y=met_object.wind_direction, mode='markers', name='Wind direction'))
fig.update_layout(height=400, margin={"r":0,"l":0}, title='Wind Direction [deg]')
fig.show(renderer="browser")

We set up a source map which contains the location information of the simulated sources. We define them in a certain location but could also let this object generate sources using for example a latin hypercube design within the specifies site limits.

In [14]:
reference_altitude

0

In [15]:
source_map = SourceMap()
site_limits = np.array([[0, 30],
                        [0, 30],
                        [0, 10]])
location_object = ENU(ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude)

source_map.generate_sources(coordinate_object=location_object, sourcemap_limits=site_limits, sourcemap_type="hypercube", nof_sources=2)

source_map.location.up = np.array([2.0, 3.0, 5.0, 8.0])
source_map.location.east = np.array([10.0, 20.0, 25.0, 29.0])
source_map.location.north = np.array([20.0, 15.0, 5.0, 1.0])

In [16]:
fig = go.Figure()
fig = sensor_group.plot_sensor_location(fig=fig)
fig.update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
fig.add_trace(
    go.Scattermap(
        mode="markers",
        lon=source_map.location.to_lla().longitude,
        lat=source_map.location.to_lla().latitude,
        name="True locations",
        marker=go.scattermap.Marker(color="green", size=10),
    )
)
fig.show(renderer="browser")

After defining the gas species we are interested in we set the true emission rates and generate the real observations. We calculate the coupling from each source to each sensor using a Gaussian plume model and multiply this coupling with the emission rates of the respective sources. We add this source contribution to the background (in this case 2.0 ppm) and also add some random (measurement) noise. These simulated observations are populating the concentration attribute of each sensor in the sensorgroup. The resulting data is shown in the plot.

In [17]:
gas_object = CH4()
dispersion_model = GaussianPlume(source_map=deepcopy(source_map))
true_emission_rates = np.array([[15], [10], [5], [7]])
for current_sensor in sensor_group.values():
    coupling_matrix = dispersion_model.compute_coupling(sensor_object=current_sensor, meteorology_object=met_object,
                                                        gas_object=gas_object, output_stacked=False, run_interpolation=False)
    source_contribution = coupling_matrix @ true_emission_rates
    observation = source_contribution.flatten() + 2.0 + np.random.normal(loc=0.0, scale=0.01, size=current_sensor.nof_observations)
    current_sensor.concentration = observation

In [18]:
fig=go.Figure()
fig = sensor_group.plot_timeseries(fig=fig)
fig.update_layout(height=800, margin={"r":0,"t":10,"l":0,"b":0})
fig.show(renderer="browser")

In [19]:
fig = go.Figure()
fig = met_object.plot_polar_scatter(fig=fig, sensor_object=sensor_group)
fig.update_layout(height=400, margin={"r":0,"l":0})
fig.show(renderer="browser")

Normally these raw datasets need some preprocessing like smoothing the data and making sure all the time axes align. Therefore we created the preprocessor class. We show the functionality to apply an aggregate function on the data within the user specified time bins and also show how to apply a wind filter, even though the wind speeds we simulated are most likely all larger than the lower limit.

In [20]:
analysis_time_range = [datetime.datetime(2024, 1, 1, 8, 0, 0), datetime.datetime(2024, 1, 1, 12, 0, 0)]

smoothing_period = 10 * 60

time_bin_edges = pd.array(pd.date_range(analysis_time_range[0], analysis_time_range[1], freq=f'{smoothing_period}s'), dtype='datetime64[ns]')

prepocessor_object = Preprocessor(time_bin_edges=time_bin_edges, sensor_object=sensor_group, met_object=met_object,
                                  aggregate_function="median")

min_wind_speed = 0.05
prepocessor_object.filter_on_met(filter_variable=["wind_speed"], lower_limit=[min_wind_speed], upper_limit=[np.inf])

In [21]:
fig=go.Figure()
fig = prepocessor_object.sensor_object.plot_timeseries(fig=fig)
fig.update_layout(height=800, margin={"r":0,"t":0,"l":0,"b":0})
fig.show(renderer="browser")

We set up a fixeed source map which contains the location of one of the sources.

In [22]:
source_map_fix_sources_1 = SourceMap()
location_object = ENU(ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude)

source_map_fix_sources_1.location = location_object
source_map_fix_sources_1.location.up = np.array([3.0])
source_map_fix_sources_1.location.east = np.array([20.0])
source_map_fix_sources_1.location.north = np.array([15.0])
dispersion_model_fix_sources_1 = GaussianPlume(source_map_fix_sources_1)


In [23]:
source_map_fix_sources_2 = SourceMap()
location_object = ENU(ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude)

source_map_fix_sources_2.location = location_object
source_map_fix_sources_2.location.up = np.array([5.0])
source_map_fix_sources_2.location.east = np.array([25.0])
source_map_fix_sources_2.location.north = np.array([5.0])
dispersion_model_fix_sources_2 = GaussianPlume(source_map_fix_sources_2)

We set up a source map for reversible jump part by generating one sources using for example a latin hypercube design within the specifies site limits.

In [24]:
source_map_rj = SourceMap()
location_object = ENU(ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude)

source_map_rj.generate_sources(coordinate_object=location_object, sourcemap_limits=site_limits, sourcemap_type="hypercube", nof_sources=2)

dispersion_model_rj = GaussianPlume(source_map_rj)


We set up the different parameters for our MCMC model.

In [25]:
source_model_fix_sources_1 = Normal(label_string="fixed_1")
source_model_fix_sources_1.emission_rate_mean = np.array([0], ndmin=1)
source_model_fix_sources_1.initial_precision = np.array([1 / (2.5 ** 2)], ndmin=1)
source_model_fix_sources_1.reversible_jump = False
source_model_fix_sources_1.rate_num_sources = 1.0
source_model_fix_sources_1.dispersion_model = dispersion_model_fix_sources_1
source_model_fix_sources_1.update_precision = False
source_model_fix_sources_1.site_limits = site_limits
source_model_fix_sources_1.coverage_detection = 0.1  # ppm
source_model_fix_sources_1.coverage_test_source = 3.0  # kg/hr

source_model_fix_sources_2 = Normal(label_string="fixed_2")
source_model_fix_sources_2.emission_rate_mean = np.array([0], ndmin=1)
source_model_fix_sources_2.initial_precision = np.array([1 / (2.5 ** 2)], ndmin=1)
source_model_fix_sources_2.reversible_jump = False
source_model_fix_sources_2.rate_num_sources = 1.0
source_model_fix_sources_2.dispersion_model = dispersion_model_fix_sources_2
source_model_fix_sources_2.update_precision = False
source_model_fix_sources_2.site_limits = site_limits
source_model_fix_sources_2.coverage_detection = 0.1  # ppm
source_model_fix_sources_2.coverage_test_source = 3.0  # kg/hr

source_model_rj = Normal()
source_model_rj.emission_rate_mean = np.array([0], ndmin=1)
source_model_rj.initial_precision = np.array([1 / (2.5 ** 2)], ndmin=1)
source_model_rj.reversible_jump = True
source_model_rj.rate_num_sources = 1.0
source_model_rj.dispersion_model = dispersion_model_rj
source_model_rj.update_precision = False
source_model_rj.site_limits = site_limits
source_model_rj.coverage_detection = 0.1  # ppm
source_model_rj.coverage_test_source = 3.0  # kg/hr

source_model = [source_model_fix_sources_1, source_model_fix_sources_2 ,source_model_rj]

background = SpatioTemporalBackground()
background.n_time = None
background.mean_bg = 2.0
background.spatial_dependence = True
background.initial_precision = 1 / np.power(3e-4, 2)
background.precision_time_0 = 1 / np.power(0.1, 2)
background.spatial_correlation_param = 25.0
background.update_precision = False

offset_model = PerSensor()
offset_model.update_precision = False
offset_model.initial_precision = 1 / (0.001)**2

error_model = BySensor()
error_model.initial_precision = 1 / (0.1)**2
error_model.prior_precision_shape = 1e-2
error_model.prior_precision_rate = 1e-2


We create an instance of the ELQModel class which forms the interface with the MCMC repo and run the MCMC algorithm. Finally we plot the results

In [26]:
elq_model = ELQModel(sensor_object=prepocessor_object.sensor_object, meteorology=prepocessor_object.met_object,
                          gas_species=gas_object, background=background,
                          error_model=error_model, offset_model=offset_model, source_model=source_model)
elq_model.n_iter = 25000

elq_model.initialise()

elq_model.to_mcmc()
elq_model.run_mcmc()
elq_model.from_mcmc()

100%|██████████| 25000/25000 [27:11<00:00, 15.32it/s]

z_src: Acceptance rate 26%
n_src: Acceptance rate 1%


In [27]:
elq_model.mcmc.store.keys()

dict_keys(['bg', 'tau', 'd', 's_fixed_1', 's_fixed_2', 's', 'z_src', 'n_src', 'y', 'log_post'])

Finally plotting the results in a separate cell. Note that you can plot all plots in 1 go as well using `plotter.show_all()`

In [ ]:
burn_in = elq_model.n_iter-12500

plotter = Plot()

plotter.plot_quantification_results_on_map(model_object=elq_model, bin_size_x=1, bin_size_y=1, normalized_count_limit=0.1, burn_in=burn_in)

plotter.plot_fitted_values_per_sensor(mcmc_object=elq_model.mcmc, sensor_object=elq_model.sensor_object, burn_in=burn_in)

true_source_location_trace = go.Scattermap(mode="markers",lon=source_map.location.to_lla().longitude,
                                              lat=source_map.location.to_lla().latitude,name="True locations",
                                              marker=go.scattermap.Marker(color="green", size=10))


In [31]:
elq_model.mcmc.store["s_fixed_1"][0, burn_in:].mean()

np.float64(9.981593140405904)

In [32]:
elq_model.mcmc.store["s_fixed_2"][0, burn_in:].mean()

np.float64(4.9751404972741335)

In [33]:
plotter.figure_dict["fitted_values"].update_layout(height=800, margin={"r":0,"t":50,"l":0,"b":0}).show(renderer="browser")

Note we could have also used this call to plot the fitted values figure:
`plotter = elq_model.plot_fitted_values(plot=plotter)`

In [34]:
plotter = elq_model.plot_fitted_values(plot=plotter)
plotter.figure_dict["fitted_values"].update_layout(height=800, margin={"r":0,"t":50,"l":0,"b":0}).show(renderer="browser")

In [35]:
plotter.figure_dict["count_map"].update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
plotter.figure_dict["count_map"].add_trace(true_source_location_trace)
plotter.figure_dict["count_map"].show(renderer="browser")

In [36]:
plotter.figure_dict["iqr_map"].update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
plotter.figure_dict["iqr_map"].add_trace(true_source_location_trace)
plotter.figure_dict["iqr_map"].show(renderer="browser")

In [37]:
plotter.figure_dict["median_map"].update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
plotter.figure_dict["median_map"].add_trace(true_source_location_trace)
plotter.figure_dict["median_map"].show(renderer="browser")

In [38]:
plotter = elq_model.plot_log_posterior(burn_in_value=burn_in, plot=plotter)
plotter.figure_dict["log_posterior_plot"].show(renderer="browser")

In [39]:
plotter = elq_model.components["source"].plot_iterations(plot=plotter, burn_in_value=burn_in, y_axis_type="linear")
plotter = elq_model.components["source"].plot_iterations(plot=plotter, burn_in_value=burn_in, y_axis_type="log")

plotter.figure_dict["estimated_values_plot"].show(renderer="browser")
plotter.figure_dict["log_estimated_values_plot"].show(renderer="browser")
plotter.figure_dict["number_of_sources_plot"].show(renderer="browser")


In [ ]:
plotter = elq_model.components["offset"].plot_iterations(plot=plotter, sensor_object=elq_model.sensor_object,burn_in_value=burn_in)
plotter = elq_model.components["offset"].plot_distributions(plot=plotter, sensor_object=elq_model.sensor_object,burn_in_value=burn_in)
plotter.figure_dict["offset_iterations"].show(renderer="browser")
plotter.figure_dict["offset_distributions"].show(renderer="browser")

In [32]:
plotter = elq_model.components["error_model"].plot_iterations(plot=plotter, sensor_object=elq_model.sensor_object,burn_in_value=burn_in)
plotter = elq_model.components["error_model"].plot_distributions(plot=plotter, sensor_object=elq_model.sensor_object,burn_in_value=burn_in)
plotter.figure_dict["error_model_iterations"].show()
plotter.figure_dict["error_model_distributions"].show()

Finally we show all keys of figures present. As said before we could have just created all figures and perform one call to `plotter.show_all()` which shows all figures at once.

In [33]:
list(plotter.figure_dict.keys())

['count_map',
 'median_map',
 'iqr_map',
 'fitted_values',
 'log_posterior_plot',
 'estimated_values_plot',
 'number_of_sources_plot',
 'log_estimated_values_plot',
 'offset_iterations',
 'offset_distributions',
 'error_model_iterations',
 'error_model_distributions']